# renaming the reactions according to columns and rows

In [34]:
from math import isclose
from json import load, dump

with open("WesternDietMicrobiomes/escherMap.json", "r") as jsonIn:    escherMap = load(jsonIn)

# display(escherMap)

xs, ys = [], []
for index, lst in enumerate(escherMap):
    for k, content in lst.items():
        if k != "reactions":  continue
        print(content)
        for rxn, content2 in content.items():
            if "label_x" not in content2:  print(content2) ; continue
            if not any([isclose(content2["label_x"], x, abs_tol=50) for x in xs]):   xs.append(content2["label_x"])
            if not any([isclose(content2["label_y"], y, abs_tol=50) for y in ys]):   ys.append(content2["label_y"])

print(xs, ys)

for index, lst in enumerate(escherMap):
    for k, content in lst.items():
        if k != "reactions":  continue
        for rxn, content2 in content.items():
            for x in xs:
                if isclose(content2["label_x"], x, abs_tol=50):
                    col_index = xs.index(x)
                    continue
            for y in ys:
                if isclose(content2["label_y"], y, abs_tol=50):
                    row_index = ys.index(y)
                    continue
            for attribute, val in content2.items():
                if attribute == "metabolites":
                    metabolite_dics = []
                    for metabolite in val:
                        metabolite_dic = {}
                        for cpd_index, (k2, val2) in enumerate(metabolite.items()):
                            if k2 == "bigg_id":  metabolite_dic[k2] = f"{val2}_{col_index}_{row_index}_{cpd_index}"
                            elif k2 == "coefficient":  metabolite_dic[k2] = val2
                        metabolite_dics.append(metabolite_dic)
                    escherMap[index][k][rxn][attribute] = metabolite_dics
                    print(escherMap[index][k][rxn][attribute])
                if attribute == "bigg_id":   escherMap[index][k][rxn][attribute] = f"{val}_{col_index}_{row_index}"
                # if attribute == "metabolites":
                    # escherMap[index][k][rxn][attribute] = val+
                # print(attribute, val)


with open("WesternDietMicrobiomes/escherMap_edited.json", "w") as jsonOut:    escherMap = dump(escherMap, jsonOut, indent=3)

{'0': {'name': 'diphosphate phosphohydrolase', 'bigg_id': 'rxn00001', 'reversibility': False, 'label_x': -642.8153489006918, 'label_y': 103.11225355934926, 'gene_reaction_rule': '', 'genes': [], 'metabolites': [{'bigg_id': 'cpd00012', 'coefficient': -1}, {'bigg_id': 'cpd00067', 'coefficient': 1}, {'bigg_id': 'cpd00001', 'coefficient': -1}, {'bigg_id': 'cpd00009', 'coefficient': 2}], 'segments': {'0': {'from_node_id': '1', 'to_node_id': '2', 'b1': None, 'b2': None}, '1': {'from_node_id': '3', 'to_node_id': '2', 'b1': None, 'b2': None}, '2': {'from_node_id': '1', 'to_node_id': '0', 'b1': {'x': -638.8122011531899, 'y': 126.74002323747482}, 'b2': {'x': -693.0594201712433, 'y': 127.28932170583562}}, '3': {'from_node_id': '3', 'to_node_id': '4', 'b1': {'x': -514.8185576833539, 'y': 125.48448388122324}, 'b2': {'x': -460.57133866530046, 'y': 124.93518541286244}}, '4': {'from_node_id': '1', 'to_node_id': '5', 'b1': {'x': -642.8119961038293, 'y': 126.78052450703285}, 'b2': {'x': -647.87947126354

# duplicating reactions across a specified number of rows and columns

In [45]:
2961.18779884681--638.8122011531899

3600.0

In [39]:
from json import load, dump

def duplicate_rxn(escherJSON_path, rows=None, columns=None, row_step_size=800, col_step_size=400):
    assert rows is not None or columns is not None, "What are you doing?"

    with open(escherJSON_path, "r") as jsonIn:
        escherMap = load(jsonIn)

    # display(escherMap)
    print(len(escherMap))

    print(row_step_size, col_step_size)

    for i, lst in enumerate(escherMap):
        if i == 0:  continue
        for k, content in lst.items():
            if k == "reactions":
                # row reaction development
                print("rows")
                print(row_step_size, col_step_size)
                updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                if isinstance(rows, int):
                    row_reactions = {k:v for k,v in content.items()}
                    for row_index in range(1, rows):
                        # print(content.keys())
                        for rxnNum, content2 in content.items():
                            nextRxn = str(int(rxnNum)+1)
                            ## define the content of this reaction
                            new_content = {k:v for k,v in content2.items()}
                            ## translate the label
                            new_content["label_y"] += row_step_size*row_index
                            ## rename the reaction according to its new translation
                            rxnName_content = new_content["bigg_id"].split("_")
                            colNum, rowNum = rxnName_content[1], rxnName_content[2]
                            new_content["bigg_id"] = f"{rxnName_content[0]}_{colNum}_{str(int(rowNum)+1)}"
                            ## rename the metabolites
                            for index, metabolite in enumerate(new_content["metabolites"]):
                                ### rename the metabolites
                                ID_content = metabolite["bigg_id"].split("_")
                                metabolite["bigg_id"] = f"{ID_content[0]}_{colNum}_{rowNum}_{index}"
                            ## update the metabolite nodes
                            display(new_content.keys())
                            ## update the edge segments
                            for num, content3 in new_content["segments"].items():
                                dictionary_values = [value for value in content3.values() if isinstance(value, dict)]
                                # print(dictionary_values)
                                # for obj, value in content3.items():
                                #     # print(list(map(len, [new_content, content3])))
                                #     if isinstance(value, dict):
                                #         print(rxnNum, num, obj, value) ;  new_content["segments"][num][obj]["y"] += row_step_size*row_index
                                #     if isinstance(value, dict):
                                #         print(new_content["segments"][num][obj])
                            row_reactions[str(int(rxnNum)+row_index)] = new_content
                            # print(row_reactions[str(int(rxn)+row_index)]["segments"]["312"]["b2"])
                        updated_reactions.update({k:v for k,v in row_reactions.items()})
                escherMap[i][k] = updated_reactions  
            elif k == "nodes":
                # row metabolite development
                print("rows")
                print(row_step_size, col_step_size)
                updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                if isinstance(rows, int):
                    row_nodes = {k:v for k,v in content.items()}
                    for row_index in range(1, rows):
                        print(content.keys())
                        # updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                        new_mets = {}
                        for metNum, content2 in row_nodes.items():
                            ### rename the node for this set of reactions
                            # print(metNum)
                            new_mets[f"{nextRxn}{metNum}"] = content2
                            ### translating the node locations
                            if new_mets[f"{nextRxn}{metNum}"]["node_type"] in ["multimarker", "midmarker"]:
                                new_mets[f"{nextRxn}{metNum}"]["y"] += row_step_size*row_index
                            elif new_mets[f"{nextRxn}{metNum}"]["node_type"] == "metabolite":
                                new_mets[f"{nextRxn}{metNum}"]["y"] += row_step_size*row_index
                                new_mets[f"{nextRxn}{metNum}"]["label_y"] += row_step_size*row_index
                        updated_reactions.update({k:v for k,v in row_reactions.items()})
                escherMap[i][k] = updated_reactions  
            # else:   print(k)
        # if index == len(escherMap)-1:  break
        # print("columns")
        # print(row_step_size, col_step_size)
        # if isinstance(columns, int):
        #     reactions = content if rows is not None else updated_reactions
        #     col_reactions = {}
        #     for col_index in range(columns):
        #         for rxnNum, content2 in reactions.items():
        #             new_content = {k:v for k,v in content2.items()}
        #             new_content["label_x"] += col_step_size*col_index
        #             for num, content3 in new_content["segments"].items():
        #                 for obj, value in content3.items():
        #                     if isinstance(value, dict):  print(rxnNum, num, obj, value) ; new_content["segments"][num][obj]["x"] += col_step_size*col_index
        #                     if isinstance(value, dict):  print(new_content["segments"][num][obj])
        #             col_reactions[str(int(rxnNum)+col_index)] = new_content
        #         updated_reactions.update({k:v for k,v in row_reactions.items()})

        #     escherMap[index][k] = updated_reactions    

    with open(escherJSON_path.replace(".json", "_edited.json"), "w") as jsonOut:  dump(escherMap, jsonOut, indent=3)

duplicate_rxn("oneRXN_localized2.json", rows=3, columns=None)

2
800 400
rows
800 400


dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

rows
800 400
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])


In [54]:
print(1596.7619819523752-396.7619819523753)
print(2796.761981952375-1596.7619819523752)

1200.0
1200.0


In [41]:
from json import load, dump

path = "oneRXN_localized_double.json"

with open(path, "r") as jsonIn:
    escherMap = load(jsonIn)

with open(path, "w") as jsonOut:
    dump(escherMap, jsonOut, indent=3)

## update the coordinates of an existing two-reaction map

In [45]:
from json import load

with open("twoRXN_localized.json", 'r') as jsonIn:
    EscherMap = load(jsonIn)

row_step_size, col_step_size = 800, 400
column = 0

new_EscherMap = {}
for i, ele in enumerate(EscherMap):
    if i == 0:
        new_EscherMap[i] = ele
    new_EscherMap[i] = {}
    for k, content in ele.items():
        # print(k)
        if k == "reactions":
            new_EscherMap[i]["reactions"] = {}
            for rxnNum, content2 in content.items():
                if rxnNum == 0:
                    new_EscherMap[i]["reactions"][rxnNum] = content2
                    continue
                for k2, val in content2.items():
                    if k == "bigg_id":
                        baseRXN = val.split("_")[0]
                        new_EscherMap[i]["reactions"][rxnNum][k2] = f"{baseRXN}_{column}_{rxnNum}"
                    elif k == "label_x":
                        new_EscherMap[i]["reactions"][rxnNum][k2] = val + column*col_step_size
                    elif k == "label_y":
                        new_EscherMap[i]["reactions"][rxnNum][k2] = val + rxnNum*row_step_size
                    elif k2 == "metabolites":
                        new_mets = []
                        for cpdNum, cpd in enumerate(val):
                            cpd["bigg_id"] = f"cpd_{column}_{rxnNum}_{cpdNum}"
                            new_mets.append(cpd)
                        new_EscherMap[i]["reactions"][rxnNum][k2] = new_mets
                    elif k2 == "segments":
                        for segNum, content3 in val.items():
                            if isinstance(content3["b1"], dict):
                                new_EscherMap[i]["reactions"][rxnNum][k2][segNum][]
                    


                display(content2)
                break

        elif k == "nodes":
            pass
        elif k == "text_labels":
            pass
        elif k == "canvas":
            pass

{'name': 'diphosphate phosphohydrolase',
 'bigg_id': 'rxn00001_0_0',
 'reversibility': False,
 'label_x': -682.8153489006918,
 'label_y': 228.44559706520863,
 'gene_reaction_rule': '',
 'genes': [],
 'metabolites': [{'bigg_id': 'cpd_0_0_0', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_1', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_2', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_3', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_4', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_5', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_6', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_7', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_8', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_9', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_10', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_11', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_12', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_13', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_14', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_15', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_16', 'coeff

# process the ASVMetaboliteInteractions community interactions

## parse the stoichiometric relationships

In [3]:
from pandas import read_csv
from json import dump

interactions = read_csv("ASVMetaboliteInteractions.csv").set_index("ASV")
stoich_mapping = {}
for index, row in interactions.iterrows():
    stoich_mapping[index] = {"reactants":{}, "products":{}}
    for col, ele in row.items():
        metID = col.split("_")[0]
        if ele > 0:   stoich_mapping[index]["products"][metID] = ele
        elif ele < 0:   stoich_mapping[index]["reactants"][metID] = ele

# display(stoich_mapping)
with open("ASV_metabolic_activity_stoich.json", 'w') as jsonOut:
    dump(stoich_mapping, jsonOut, indent=3)

## process the stoich into Escher mapping coordinates

# attempt loading the ASV-effective model

In [94]:
from cobra.io import load_json_model
from optlang import Objective

effectiveASV_model = load_json_model("ASVInteractionModel.json")
rxnNames = [rxn.id for rxn in effectiveASV_model.reactions]
rcNames = {}
numRCs = []
for name in rxnNames:
    if "RC" in name:
        nameParts = name.split("RC")
        numRCs.append(nameParts[0])
        rcNames[nameParts[0]] = rcNames.get(nameParts[0], [])
        rcNames[nameParts[0]].append(nameParts[1].split("_")[1])
print(set(rcNames.keys()) - set(numRCs))
wdNames = {}
numWDs = []
for name in rxnNames:
    if "WD" in name:
        nameParts = name.split("WD")
        numWDs.append(nameParts[0])
        wdNames[nameParts[0]] = wdNames.get(nameParts[0], [])
        wdNames[nameParts[0]].append(nameParts[1].split("_")[1])
print(set(wdNames.keys()) - set(numWDs))
new_rcNames = [f"{k}RC-ABX_{time}" for k,v in rcNames.items() for time in v]
new_wdNames = [f"{k}WD-ABX_{time}" for k,v in wdNames.items() for time in v]

missing_keys = set(rxnNames) - set(new_rcNames+new_wdNames)
print(missing_keys)

def search_days(day):
    RCasvs = [asv for asv, days in rcNames.items() if day in days]
    WDasvs = [asv for asv, days in wdNames.items() if day in days]
    display("RC", sorted(RCasvs))
    display("WD", sorted(WDasvs))

search_days("9.0")

set()
set()
set()


'RC'

['Acetivibrio.1',
 'Akkermansia.1',
 'Alistipes.2',
 'Alistipes.4',
 'Bifidobacterium.1',
 'Blautia.1',
 'Clostridium.16',
 'Clostridium.21',
 'Clostridium.23',
 'Enterococcus.2',
 'Enterococcus.3',
 'Eubacterium.7',
 'Lactobacillus.1',
 'Lactobacillus.3',
 'Merdibacter.1',
 'Muribaculum.1',
 'Ruminococcus.3',
 'Turicibacter.1']

'WD'

['Acinetobacter.1',
 'Agrobacterium.1',
 'Enterococcus.1',
 'Enterococcus.2',
 'Lactobacillus.3',
 'Lactococcus.1',
 'Stenotrophomonas.1']

# define new metabolite categories

In [95]:
from modelseedpy.biochem import from_local

msdb = from_local("../../ModelSEEDDatabase")

In [158]:
# from deepdiff import DeepDiff
from numpy import array, load, append, save, concatenate, setdiff1d

scfa = load("../../CommScores/commscores/data/categories/scfa.npy")
vitamins = load("../../CommScores/commscores/data/categories/vitamins.npy")
aminoAcids = load("../../CommScores/commscores/data/categories/aminoAcids.npy")
sugars = load("../../CommScores/commscores/data/categories/sugars.npy")
# complex_sugars = array([["cpd00158", "CELB"], ["cpd00382", "Melitose"]])
# save("../../CommScores/commscores/data/categories/complex_sugars.npy", complex_sugars)
# sugars_dict = dict(sugars)
# new_sugars_dict = {}
# for cpdID, name in sugars_dict.items():
#     if name not in ["CELB", "Melitose"]:
#         new_sugars_dict[cpdID] = name
# sugars = array(list(zip(new_sugars_dict.keys(), new_sugars_dict.values())))
# save("../../CommScores/commscores/data/categories/sugars.npy", sugars)
# print(DeepDiff(sugars_dict, new_sugars_dict))
complex_sugars = load("../../CommScores/commscores/data/categories/complex_sugars.npy")
# new_sugars = setdiff1d(sugars, complex_sugars)
# display(new_sugars)
# new_sugars = subtract(sugars, array([["cpd00158", "CELB"], ["cpd00382", "Melitose"]]))
# display(new_sugars)
# save("../../CommScores/commscores/data/categories/sugars.npy", new_sugars)
energy_compounds = load("../../CommScores/commscores/data/categories/energy_compounds.npy")

In [162]:
from numpy import save, delete
fatty_acids = [["cpd01113", "Hexanoate"], ["cpd03846", "Octanoate"], ["cpd01107", "Caprate"], ["cpd01741", "Laurate"],
               ["cpd15640", "Tridecanoate"], ["cpd03847", "Myristate"], ["cpd16351", "Pentadecanoate"], ["cpd00214", "Palmitate"],
               ["cpd15609", "heptadecanoate"], ["cpd01080", "Stearate"], ["cpd00536", "Oleate"],
               ["cpd01122", "Linoelate"], ["cpd03850", "Linoleate"], ["cpd03848", "Arachidate"]]
save("../../CommScores/commscores/data/categories/fattyAcids.npy", fatty_acids)

In [112]:
display(aminoAcids)

array([['cpd00035', 'Alanine'],
       ['cpd00051', 'L-Arginine'],
       ['cpd00132', 'L-Asparagine'],
       ['cpd00041', 'L-Aspartate'],
       ['cpd00084', 'L-Cysteine'],
       ['cpd00023', 'L-Glutamate'],
       ['cpd00053', 'L-Glutamine'],
       ['cpd00033', 'Glycine'],
       ['cpd00119', 'L-Histidine'],
       ['cpd00747', 'L-Hydroxyproline'],
       ['cpd00322', 'L-Isoleucine'],
       ['cpd00107', 'L-Leucine'],
       ['cpd00039', 'L-Lysine'],
       ['cpd00060', 'L-Methionine'],
       ['cpd00066', 'L-Phenylalanine'],
       ['cpd00129', 'L-Proline'],
       ['cpd00054', 'L-Serine'],
       ['cpd00161', 'L-Threonine'],
       ['cpd00065', 'L-Tryptophan'],
       ['cpd00069', 'L-Tyrosine'],
       ['cpd00156', 'L-Valine']], dtype='<U16')

In [164]:
# define the sets of amino acids
aromaticAA = array([['cpd00066', 'L-Phenylalanine'], ['cpd00069', 'L-Tyrosine'], ['cpd00065', 'L-Tryptophan']])
pyruvateAA = array([['cpd00035', 'Alanine'], ['cpd00107', 'L-Leucine'], ['cpd00322', 'L-Isoleucine'], ['cpd00156', 'L-Valine']])
alphaKetaGlutarateAA = array([['cpd00023', 'L-Glutamate'], ['cpd00053', 'L-Glutamine'], ['cpd00129', 'L-Proline'], ['cpd00051', 'L-Arginine'],
                              ['cpd00747', 'L-Hydroxyproline']])
oxaloacetateAA = array([['cpd00132', 'L-Asparagine'], ['cpd00041', 'L-Aspartate'], 
                        ['cpd00060', 'L-Methionine'], ['cpd00161', 'L-Threonine'], ['cpd00039', 'L-Lysine']])
phosphoglycerateAA = array([['cpd00054', 'L-Serine'], ['cpd00033', 'Glycine'], ['cpd00084', 'L-Cysteine'], ["cpd00550", "D-Serine"]])
oxoAAs = array([["cpd01293", "Oxoproline"], ["cpd01292", "Oxoproline"], ["cpd00024", "2-Oxoglutarate"]])

# save the amino acid categories
save("../../CommScores/commscores/data/categories/aromaticAA.npy", aromaticAA)
save("../../CommScores/commscores/data/categories/pyruvateAA.npy", pyruvateAA)
save("../../CommScores/commscores/data/categories/alphaKetaGlutarateAA.npy", alphaKetaGlutarateAA)
save("../../CommScores/commscores/data/categories/oxaloacetateAA.npy", oxaloacetateAA)
save("../../CommScores/commscores/data/categories/phosphoglycerateAA.npy", phosphoglycerateAA)
save("../../CommScores/commscores/data/categories/oxoAAs", oxoAAs)

In [165]:
from numpy import concatenate

all_specifiedAAs = concatenate([aromaticAA, pyruvateAA, alphaKetaGlutarateAA, oxaloacetateAA, phosphoglycerateAA])
for aa in aminoAcids:
    if aa not in all_specifiedAAs:
        print(aa)

['cpd00119' 'L-Histidine']


In [147]:
scfa
# new_energy_compounds = []
# for cpd, name in energy_compounds:
#     # print(cpd, name)
#     if name in fatty_acids:
#         new_energy_compounds = delete(energy_compounds, name, 1)

array([['cpd00029', 'Acetate'],
       ['cpd00141', 'Propionate'],
       ['cpd00211', 'Butyrate'],
       ['cpd01711', 'Isobutyrate'],
       ['cpd00597', 'Valerate'],
       ['cpd16265', 'M_2_Keto_isovalerate'],
       ['cpd35499', '(S)-2-methylbutanoate'],
       ['cpd00106', 'Fumarate']], dtype='<U21')

In [111]:
print(len(new_energy_compounds), len(energy_compounds))

0 29614


In [104]:
new_scfa = append(scfa, [["cpd00106", "Fumarate"]], axis=0)
display(new_scfa)

array([['cpd00029', 'Acetate'],
       ['cpd00141', 'Propionate'],
       ['cpd00211', 'Butyrate'],
       ['cpd01711', 'Isobutyrate'],
       ['cpd00597', 'Valerate'],
       ['cpd16265', 'M_2_Keto_isovalerate'],
       ['cpd35499', '(S)-2-methylbutanoate'],
       ['cpd00106', 'Fumarate']], dtype='<U21')

In [168]:
from json import load

with open("ASVInteractionModel.json", 'r') as jsonIn:
    model = load(jsonIn)

# check which vitamins are in the model
# metabolites = [met["id"] for met in model["metabolites"]]
# vitamin_dict = dict(vitamins)
# for metID in metabolites:
#     if metID in vitamin_dict:
#         print(metID)

# define dictionaries of all the categories
## amino acids
aromaticAA_dict, pyruvateAA_dict, alphaKetaGlutarateAA_dict = dict(aromaticAA), dict(pyruvateAA), dict(alphaKetaGlutarateAA)
oxaloacetateAA_dict, phosphoglycerateAA_dict, oxoAA_dict = dict(oxaloacetateAA), dict(phosphoglycerateAA), dict(oxoAAs)
## fatty acids
fatty_acids_dict, scfa_dict, sugars_dict, complex_sugars_dict = dict(fatty_acids), dict(scfa), dict(sugars), dict(complex_sugars)

# categorize the model metabolites
not_changed = []
for met in model["metabolites"]:
    ## amino acids
    if met["id"] in aromaticAA_dict:   met["id"] = "Aromatic AA"
    elif met["id"] in pyruvateAA_dict:   met["id"] = "Pyruvate AA"
    elif met["id"] in alphaKetaGlutarateAA_dict:   met["id"] = "alphaKetaGlutarate AA"
    elif met["id"] in oxaloacetateAA_dict:   met["id"] = "Oxaloacetate AA"
    elif met["id"] in phosphoglycerateAA_dict:   met["id"] = "phosphoGlycerate AA"
    elif met["id"] in oxoAA_dict:   met["id"] = "oxoAA"
    ## other compounds
    elif met["id"] in fatty_acids_dict:   met["id"] = "fatty Acid"
    elif met["id"] in scfa_dict:   met["id"] = "scFatty Acid"
    elif met["id"] in sugars_dict:   met["id"] = "Sugar"
    elif met["id"] in complex_sugars_dict:   met["id"] = "complex Sugar"
    else:   not_changed.append(met["id"])

print(f"{len(not_changed)}/{len(model['metabolites'])} compounds are not yet exchanged")
print(not_changed)

# export the converted model
from json import dump
print(model["metabolites"])
with open("ASVInteractionModel_categorizedIDs.json", 'w') as jsonOut:
    dump(model, jsonOut, indent=3)

18/63 compounds are not yet exchanged
['cpd35610', 'cpd00851', 'cpd07053', 'cpd02475', 'cpd04945', 'cpd00122', 'cpd00526', 'cpd00137', 'cpd02733', 'cpd00119', 'cpd00359', 'cpd00121', 'cpd00130', 'cpd00392', 'cpd00218', 'cpd00064', 'cpd00020', 'cpd00036']
[{'id': 'cpd35610', 'name': '3alpha-hydroxy-7-oxo-5beta-cholan-24-oate', 'compartment': ''}, {'id': 'cpd00851', 'name': 'trans-4-Hydroxy-L-proline', 'compartment': ''}, {'id': 'oxoAA', 'name': '5-Oxoproline', 'compartment': ''}, {'id': 'cpd07053', 'name': 'Picolinic acid', 'compartment': ''}, {'id': 'cpd02475', 'name': 'Lithocholate', 'compartment': ''}, {'id': 'cpd04945', 'name': 'Ursodiol', 'compartment': ''}, {'id': 'scFatty Acid', 'name': 'Acetate', 'compartment': ''}, {'id': 'cpd00122', 'name': 'N-Acetyl-D-glucosamine', 'compartment': ''}, {'id': 'oxoAA', 'name': '2-Oxoglutarate', 'compartment': ''}, {'id': 'Pyruvate AA', 'name': 'L-Alanine', 'compartment': ''}, {'id': 'Sugar', 'name': 'D-Allose', 'compartment': ''}, {'id': 'Sugar

In [220]:
from json import load
with open("EscherMap_v1_categorized.json", 'r') as jsonIn:
    escherMap = load(jsonIn)

og_escherMap = [ele for ele in escherMap]

from collections import Counter
# display(escherMap)
newEscher = []
for index, ele in enumerate(og_escherMap):
    og_ele = {k:v for k,v in ele.items()}
    if index == 0:  newEscher = [ele]
    newEscher.append({})
    for k, val in ele.items():
        if k == "reactions":
            newEscher[index][k] = {}
            reactions = {}
            for rxnNum, content in val.items():
                # display(len(content["metabolites"]), content["metabolites"])
                # print(len(content["metabolites"]))
                # create a mapping from IDs to their respective coefficients
                metContents = {met["bigg_id"]: met["coefficient"] for met in content["metabolites"]}
                names = {}
                for ID in metContents.keys():
                    name = idName[ID]
                    if name in names: names[name].append(ID)
                    else: names[name] = [ID]
                # names = Counter([idName[ID]:ID for ID in metContents.keys()])
                # determine the ID with the highest coefficient, which will be exclusively kept in the final Escher
                new_met_content = []
                for k2,v in names.items():
                    if len(v) == 1:
                        new_met_content.append({"bigg_id": v[0], "coefficient": metContents[v[0]]})
                    elif len(v) > 1:
                        ReactantWinner, ProductWinner = [None, 0], [None, 0]
                        for ID, coef in metContents.items():
                            if ID in v:
                                if coef > 0 and coef > ProductWinner[1]:
                                    ProductWinner = [ID, coef]
                                if coef < 0 and coef < ReactantWinner[1]:
                                    ReactantWinner = [ID, coef]
                        for met in content["metabolites"]:
                            if met["bigg_id"] in v and met["bigg_id"] in [ReactantWinner[0], ProductWinner[0]]:
                                new_met_content.append(met)
                print(len(new_met_content), len(content["metabolites"]))
                # update the new Escher Figure
                newEscher[index][k][rxnNum] = {}
                for key, value in content.items():
                        # print(index, k, rxnNum, key, value)
                        # if key == "metabolites":
                        #     print("here")
                        #     print(len(value) == len(new_met_content))
                        newEscher[index][k][rxnNum][key] = new_met_content if key == "metabolites" else value
                # escherMap[index]["reactions"][rxnNum]["metabolites"] = new_met_content
                # print(len(content["metabolites"]) == len(newEscher[index][k][rxnNum]["metabolites"]))
                # display(len(new_met_content), escherMap[index]["reactions"][rxnNum]["metabolites"])
                # break
        else:
            newEscher[index][k] = val
    # print(DeepDiff(og_ele, newEscher[index]))

from deepdiff import DeepDiff
display(DeepDiff(og_escherMap[1], newEscher[1]))

from json import dump
with open("EscherMap_v1_categorized_simplified.json", 'w') as jsonOut:
    dump(escherMap, jsonOut, indent=3)

15 19
10 12
10 14
12 15
11 12
16 23
13 17
12 14
13 16
12 16
15 19
14 18
12 15
15 19
12 18
10 14
11 16
13 17
11 14
16 21
12 16
16 20
13 17
11 12
13 19
15 23
10 12
18 22
15 21
18 25
13 16
18 22
12 14
15 17
10 11
14 18
14 17
9 10
14 16
11 13
19 23
9 12
13 15
14 14
9 10
13 13
17 22
16 30
14 19
13 15
14 18
15 21
11 13
11 14
12 17
13 16
19 34
9 15
12 15
9 10
12 13
13 17
7 10
12 17
10 14
12 16
14 17
17 28
14 19
14 15
17 19
12 15
14 17
13 16
16 19
11 12
12 16
9 14
11 14
8 10
8 9
13 15
10 10
7 8
14 20
10 14
14 19
8 9
9 11
16 18
14 20
10 12
13 17
12 15
13 18
11 14
10 14
8 10
10 14
13 19
12 15
6 8
12 16
14 17
7 9
8 11
5 7
17 21
4 6
10 14
16 19
12 13
16 22
10 11
15 18


{'values_changed': {"root['reactions']['0']['metabolites'][1]['bigg_id']": {'new_value': 'cpd00211',
   'old_value': 'cpd00035'},
  "root['reactions']['0']['metabolites'][1]['coefficient']": {'new_value': -5.0150517867793925,
   'old_value': 0.7828477289514637},
  "root['reactions']['0']['metabolites'][2]['bigg_id']": {'new_value': 'cpd00035',
   'old_value': 'cpd00041'},
  "root['reactions']['0']['metabolites'][2]['coefficient']": {'new_value': 0.7828477289514637,
   'old_value': 0.7236527848953159},
  "root['reactions']['0']['metabolites'][3]['bigg_id']": {'new_value': 'cpd00107',
   'old_value': 'cpd00060'},
  "root['reactions']['0']['metabolites'][3]['coefficient']": {'new_value': -0.0814322208559873,
   'old_value': -0.1333458873983672},
  "root['reactions']['0']['metabolites'][4]['bigg_id']": {'new_value': 'cpd00041',
   'old_value': 'cpd00064'},
  "root['reactions']['0']['metabolites'][4]['coefficient']": {'new_value': 0.7236527848953159,
   'old_value': 0.2787375129508895},
  "

In [176]:
idName = {content["bigg_id"]: content["name"] for content in escherMap[1]["nodes"].values() if "bigg_id" in content}

from json import dump
with open("EscherNodeMapping.json", 'w') as jsonOut:
    dump(idName, jsonOut, indent=3)